In [ ]:
from nbtemplate import display_header, get_path
display_header('CoatedGratings.ipynb')

In [ ]:
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.table import QTable, join
import astropy.units as u

from marxs.source import PointSource, FixedPointing, JitterPointing
from marxs.analysis import resolvingpower_from_photonlist
from marxslynx.simulations import run_monoenergetic_simulation
from marxs.simulator import Sequence
from marxs.optics.grating import CATGrating
from marxslynx.ralfgrating import facet_table, order_selector_Si, order_selector_SiPt

%matplotlib inline
import matplotlib.pyplot as plt

from marxslynx import AXIS as axis

In [ ]:
instrum = axis.PerfectAXIS()

# Comparing Pure Si CAT gratings with coated gratings

## Dispersed spectrum

In [ ]:
wavegrid = np.arange(1.5, 5., 0.1) * u.nm
energy = wavegrid.to(u.keV, equivalencies=u.spectral())

In [ ]:
def res_power_angle(photons, subaperangle, instrum, orders, ang_0=0):
    resolvingpower = np.zeros((len(subaperangle), len(orders)))
    aeff_per_order = np.zeros_like(resolvingpower)
    for i, ang in enumerate(subaperangle):
        ind = np.abs(np.abs(photons['facet_ang']) - ang_0) < ang
        res, width, pos = resolvingpower_from_photonlist(photons[ind], orders, zeropos=0, col='projcirc_y')
        resolvingpower[i, :] = res
        aeff_per_order[i, :] = [photons['probability'][ind & (photons['order'] == o)].sum() for o in orders]
    aeff_per_order = aeff_per_order * instrum.elements[0].area.to(u.cm**2) / photons.meta['EXPOSURE'][0]
    return resolvingpower, aeff_per_order

In [ ]:
tsubaperangle=np.linspace(0, np.pi/2, 7)[1:]

def run_multi_energy(instrum, energy, orders, subaperangle=np.linspace(0, np.pi, 7)[1:],
                    tsubaperangle=tsubaperangle):
    facettab = facet_table(instrum.elements[4])
    phot_en = []

    for i, e in enumerate(energy):
        n = 1e4 * u.s
        if e.value > 1.25:
            n=5e4 * u.s
    
        p = run_monoenergetic_simulation(instrum, e, n)
        p = join(p, facettab)
        phot_en.append(p)
        
    for p in phot_en:
        ind = p['CCD_ID'] >= 0
        trespow, taeff = res_power_angle(p[ind], tsubaperangle, instrum, orders, np.pi/2)
        p.trespow = trespow
        p.taeff = taeff
        respow, aeff = res_power_angle(p[ind], subaperangle, instrum, orders)
        p.respow = respow
        p.aeff = aeff   
    return phot_en

In [ ]:
orders = order_selector_Si.orders

# Now run with uncoated gratings
for e in instrum.elements_of_class(CATGrating):
    e.order_selector = order_selector_Si
    
phot_Si = run_multi_energy(instrum, energy, order_selector_Si.orders)

In [ ]:
# Now run with Pt coated gratings
for e in instrum.elements_of_class(CATGrating):
    e.order_selector = order_selector_SiPt
    
phot_Pt = run_multi_energy(instrum, energy, order_selector_SiPt.orders)

In [ ]:
def ang_aeff_res(phot_en, subaperangle, energy, orders):
    resolvingpower_en = np.zeros((len(subaperangle), len(energy)))

    for i, e in enumerate(energy):
        p = phot_en[i]
        zeropos = np.mean(p['detcirc_phi'][p['order']==0])
        resolvingpower = np.zeros((len(subaperangle), len(orders)))
        for j, ang in enumerate(subaperangle):
            ind = p['CCD_ID'] >=0
            res, width, pos = resolvingpower_from_photonlist(p[np.abs(p['facet_ang']) < ang],
                                                             orders, col='detcirc_phi', zeropos=zeropos)
            resolvingpower[j, :] = res
        
        resolvingpower = np.ma.masked_invalid(resolvingpower)
        # Mask out the orders 0 and 1 which always havwe low resolving power 0 
        resolvingpower[:, np.abs(orders) < 2] = np.ma.masked
        res = np.ma.average(resolvingpower, axis=1, 
                            weights=order_selector_Si.probabilities([e], [0], [axis.conf['blazeang']])[1].flatten())
        resolvingpower_en[:, i] = res
    
    en_trespos = np.stack([p.trespow for p in phot_en])
    en_taeff = np.stack([p.taeff for p in phot_en])
    # There are Nans in there which srew up the average
    en_trespos[np.isnan(en_trespos)] = 0
    
    return en_taeff, en_trespos

In [ ]:
en_taeff, en_trespos = ang_aeff_res(phot_Si, subaperangle, energy, orders)
en_taeffPt, en_tresposPt = ang_aeff_res(phot_Pt, subaperangle, energy, orders)

In [ ]:

fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for i, ang in enumerate(tsubaperangle):
    out = ax1.plot(wavegrid, en_taeff[:, i, :].sum(axis=1), label='{:3.0f}'.format(np.rad2deg(ang)))
    ax1.plot(wavegrid, en_taeffPt[:, i, :].sum(axis=1), label='{:3.0f}'.format(np.rad2deg(ang)),
            ls=':', color=out[0].get_color())
#ax.legend(title='Subaperture\nangle [deg]')
ax1.set_ylabel('Effective Area [cm$^2$]')
ax1.set_xlabel('wavelength [nm]')
ax1.set_xlim([1, 5])

for i, ang in enumerate(tsubaperangle):
    out = ax2.plot(wavegrid, np.average(en_trespos[:, i, :], axis=1, weights=en_taeff[:, i,:]), 
             label='{:3.0f} %'.format(np.rad2deg(ang)*4/360*100))
    out = ax2.plot(wavegrid, np.average(en_tresposPt[:, i, :], axis=1, weights=en_taeffPt[:, i,:]), 
                  label='__no_label__',
                  ls=':', color=out[0].get_color())
ax2.legend(title='Aperture area covered by gratings', ncol=2, loc='lower left')
ax2.set_ylabel('Resolving power')
ax2.set_xlabel('wavelength [nm]')
ax2.set_ylim([2000, None])
ax2.set_xlim([1, 5])

fig.subplots_adjust(wspace=.3)

Effective area and resolving power for simulations with different sub-aperturing. Solid lines are for pure Si gratings, dotted lines are for Pt coated gratings. Coated gratings extend the useful range to lower wavelengths (higher energies) but they are not as efficient aroudn 2 nm (about O VII triplet). This could be compensated by filling a larger fraction of the aperture with gratings, which in turn would reduce the resolving power slightly (unless we chirp or bend the gratings, see notebook on different grating sizes and properties). We could also use a mixture of Si and Pt-coated gratings to retain some effective area at lower energies, while not looring to much effective area around O VII.

## The zeroth order

The presence of the CAT gratings reduces the signal detected in the zeroth order. Some photons are dispersed to higher orders (after all, that is why we use the gratings in the first place), but there is also a loss due to absorption by the grating frames, the mounting structure, the support structure that is part of the grating membrane, and also the grating bars themselves. For high energies, Si becomes transparent, but the grating still disperse some signal because they act as phase shifting gratings at that point.

On the other hand, the signal detected in the zeroth order is scientifically valuable. If sufficient light passes through the gratings, soft X-rays can be analyzed in the grating spectrometer at the same time as the CCD spectrum of the high-energy photons is analyzed in the imaging detector.

In [ ]:
energybins = np.arange(.1, 10, .1) * u.keV
energymidpoints = 0.5 * (energybins[:-1] + energybins[1:])

In [ ]:
mysource = PointSource(coords=SkyCoord(0., 0., unit='deg'),
                           energy=QTable({"energy": energybins, 
                                   "fluxdensity": np.ones(len(energybins)) / u.s / u.cm**2 / u.keV}),
                      )
fixedpointing = FixedPointing(coords=SkyCoord(0., 0., unit='deg'))
photons = mysource.generate_photons(2e5 * u.s)
photons = fixedpointing(photons)

In [ ]:
instrum_preGAS = Sequence(elements=instrum.elements[:4])
instrum_GAS = Sequence(elements=instrum.elements[4:])

In [ ]:
photons = instrum_preGAS(photons)

In [ ]:
# Now run with uncoated gratings
for e in instrum_GAS.elements_of_class(CATGrating):
    e.order_selector = order_selector_Si
    
p = instrum.elements[4](photons.copy())

In [ ]:
# Now run the same with Pt coated gratings
for e in instrum_GAS.elements_of_class(CATGrating):
    e.order_selector = order_selector_SiPt

ppt = instrum.elements[4](photons.copy())

In [ ]:
before = np.histogram(photons['energy'], weights=photons['probability'], bins=energybins)
ind = p['order'] == 0
after = np.histogram(p['energy'][ind], weights=p['probability'][ind], bins=energybins)
ind = ppt['order'] == 0
afterpt = np.histogram(ppt['energy'][ind], weights=ppt['probability'][ind], bins=energybins)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
frac = after[0] / before[0]
fracpt = afterpt[0] / before[0]
tsubaperangle = np.linspace(0, np.pi/2, 7)[1:]
for i, ang in enumerate(tsubaperangle):
    line, = ax.plot(energymidpoints, 
             1 - ((2 * ang)/np.pi * frac + (np.pi -(2*ang))/np.pi), 
             label='{:3.0f} %'.format(np.rad2deg(ang)*4/360*100))
    ax.plot(energymidpoints, 
            1 - ((2 * ang)/np.pi * fracpt + (np.pi -(2 * ang))/np.pi), 
            ls=':', color=line.get_color())
ax.legend(title='Aperture area\ncovered\nby gratings', ncol=1, loc='upper right')
ax.set_xlabel('energy [keV]')
ax.set_ylabel('fraction of photons removed from\nbeam when XGS is inserted')
ax.set_ylim([0, 1.])
ax.set_xlim([0, 16.])
fig.savefig(get_path('figures') + '/highen.png', 
            dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/highen.pdf', bbox_inches='tight')

This plot shows how much of the incoming signal is removed from the beam by the CAT gratings. The solid lines are for pure Si gratings, the dotted lines for Si gratings coated with Pt. This increases the grating efficiency between 1 and 2 keV, but is also reduces the signal seen in zeroth order. Different colors represent different filling factors.  70% of the high-energy signal would still be available for the zero-order detector, even if we cover 2/3 of the aperture with CAT gratings. This fraction decreases for lower energies, in particular when using PT coated gratings, but below about 2 keV the dispersed signal in the grating spectrometer is more valuable than the zeroth-order signal anyway. Unless the entire aperture is filled with gratings, there will still be sufficient signal at the zeroth order to determine the position, even for very soft sources.